In [47]:
%serialconnect


Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [34]:
%sendtofile config.txt

pinled           2
deviceletter     @
connection0      JulianS9,beckaaaa,192.168.43.1,9042
analogreadpin0   t4
analogreadpin1   t13
analogreadrate   100


Sent 6 lines (153 bytes) to config.txt.


In [48]:
%sendtofile config.txt

pinled           2
deviceletter     D
connection0      JulianS9,beckaaaa,192.168.43.1,9042
analogreadpin0   t4
analogreadpin1   t13
analogreadrate   100


Sent 6 lines (153 bytes) to config.txt.


In [18]:
%sendtofile --source UDPblackviewphone.py
%sendtofile --source utils.py
# Use deviceletter=E for second ascii stream (to decode in the Android code)

Sent 32 lines (969 bytes) to utils.py.


In [43]:
%sendtofile main.py

from machine import Pin, ADC, TouchPad
import time, socket, urandom, machine
from UDPblackviewphone import connectActivePhone, si

fconfig = dict(x.strip().split(None, 1)  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
deviceletter = fconfig["deviceletter"]
c = fconfig["connection0"].split(",")
hotspots = { c[0].encode(): (c[1].encode(), c[2], int(c[3])) }
androidipnumber = c[2]
portnumber = int(c[3])
print("deviceletter", deviceletter)

while not connectActivePhone(pinled, hotspots):
    for j in range(3):
        for i in range(7):
            pinled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

def apinfunc(k):
    if k[0] == "t":
        return TouchPad(Pin(int(k[1:])))
    return ADC(Pin(int(k)))

apin0 = apinfunc(fconfig["analogreadpin0"])
apin1 = apinfunc(fconfig["analogreadpin1"])
areadrate = int(fconfig["analogreadrate"])

nbs = bytearray("Nt00000000s000000r000000\n")
mnbs = memoryview(nbs)

tstamparead = time.ticks_ms()
i = 0
while True:
    try:
        print("making socket")
        try:
            ss = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        except OSError as e:
            print("Socket can't be created")
            break
        ss.settimeout(1)
        print("socket is", ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        dwrite = s.write
        while True:
            tstamp = time.ticks_ms()
            if tstamp > tstamparead:
                tstamparead = tstamp + areadrate
                mnbs[2:10] = b"%08X" % tstamp
                mnbs[11:17] = b"%06X" % apin0.read()
                mnbs[18:24] = b"%06X" % apin1.read()
                dwrite(nbs)
                time.sleep_ms(1)
                i += 1
                if (i%50) == 1:
                    print(nbs)
                pinled.value((i%10) == 1)
                
    except OSError as e:
        msg = str(e)
        if e.args[0] == 118:
            msg = "wifi not connected"
        elif e.args[0] == 23:
            msg = "socket error"
        print("OSError", msg)
        time.sleep(2)
        
    if time.ticks_ms() - tstamparead > 20000:
        print("Quitting loop after 20 seconds of failure")
        break
        
print("All else fails, try reset")
machine.reset()


Sent 78 lines (2445 bytes) to main.py.


In [36]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.17 on 2021-09-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f7a41579760, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
